### Path Planning

Through the rest of this notebook, we'll be exploring different steps in creating an object detection algorithm. By the end, you should know how to use a pre-trained YOLO model to:

- Identify objects
- Create bounding boxes
- Count total detected
- Label individual items

#### 1. Implementing a Path Planning Algorithm

##### 1.1 Import Necessary Libraries and Helpers

Let's import the necessary libraries and helpers. The display function will help you display the images you create using the results from your model. This notebook is particularly reliant on the helper functions to make your life easier so make sure to read through those and understand what they do.

In [13]:
import pygame
import sys
from math import sqrt

In [15]:
# Constants
WINDOW_WIDTH = 1000
WINDOW_HEIGHT = 1000
CENTER_X = WINDOW_WIDTH // 2
CENTER_Y = WINDOW_HEIGHT // 2
BUTTON_WIDTH = 100
BUTTON_HEIGHT = 50
BUTTON_X = 20
BUTTON_Y = 20
BUTTON_COLOR = (0, 255, 0)
TEXT_COLOR = (0, 0, 0)
LINE_COLOR = (0, 74, 173)
POINT_COLOR = (255, 189, 89)
LABEL_COLOR = (0, 0, 0)
POINT_RADIUS = 5
ORIGINAL_WIDTH = 178.308
ORIGINAL_HEIGHT = 178.308
SCALE_FACTOR_X = (WINDOW_WIDTH / 2) / ORIGINAL_WIDTH
SCALE_FACTOR_Y = (WINDOW_HEIGHT / 2) / ORIGINAL_HEIGHT

MATCH_LOAD_REF_X = 95.00
MATCH_LOAD_REF_Y = 153.010875
MATCH_LOAD_CENTER_XY = 124.005438
GOAL_ZONE_XY = 58.334275

predefined_lines = [
    ((500, 167), (500, 833)),
    ((334, 167), (666, 167)),
    ((334, 833), (666, 833)),
    ((0, 345), (210, 345)),
    ((0, 675), (210, 675)),
    ((210, 345), (210, 675)),
    ((1000, 345), (790, 345)),
    ((790, 345), (790, 675)),
    ((1000, 655), (790, 675)),
]

points = [
    (MATCH_LOAD_REF_X * SCALE_FACTOR_X, MATCH_LOAD_REF_Y * SCALE_FACTOR_Y),
    (MATCH_LOAD_CENTER_XY * SCALE_FACTOR_X, MATCH_LOAD_CENTER_XY * SCALE_FACTOR_Y),
    (GOAL_ZONE_XY * SCALE_FACTOR_X, GOAL_ZONE_XY * SCALE_FACTOR_Y),
    (GOAL_ZONE_XY * SCALE_FACTOR_X, -GOAL_ZONE_XY * SCALE_FACTOR_Y),
    (MATCH_LOAD_CENTER_XY * SCALE_FACTOR_X, -MATCH_LOAD_CENTER_XY * SCALE_FACTOR_Y),
    (MATCH_LOAD_REF_X * SCALE_FACTOR_X, -MATCH_LOAD_REF_Y * SCALE_FACTOR_Y),
    (-MATCH_LOAD_REF_X * SCALE_FACTOR_X, -MATCH_LOAD_REF_Y * SCALE_FACTOR_Y),
    (-MATCH_LOAD_CENTER_XY * SCALE_FACTOR_X, -
     MATCH_LOAD_CENTER_XY * SCALE_FACTOR_Y),
    (-GOAL_ZONE_XY * SCALE_FACTOR_X, -GOAL_ZONE_XY * SCALE_FACTOR_Y),
    (-GOAL_ZONE_XY * SCALE_FACTOR_X, GOAL_ZONE_XY * SCALE_FACTOR_Y),
    (-MATCH_LOAD_CENTER_XY * SCALE_FACTOR_X,
     MATCH_LOAD_CENTER_XY * SCALE_FACTOR_Y),
    (-MATCH_LOAD_REF_X * SCALE_FACTOR_X, MATCH_LOAD_REF_Y * SCALE_FACTOR_Y)
]


In [14]:
# Helper Functions DO NOT CHANGE
def find_point_index(point, points):
    try:
        return points.index(point)
    except ValueError:
        return -1  # Return -1 if the point is not found in the list


def handle_intersection(clicked_points, points, lines, window, LINE_COLOR):
    start = find_nearest_point(clicked_points[0], points)
    end = find_nearest_point(clicked_points[1], points)
    path_a = get_path_points(start, end, points)
    reverse = points[::-1]
    path_b = get_path_points(start, end, reverse)

    if len(path_a) > len(path_b):
        path = path_b
        button_text = "Path B"
    else:
        path = path_a
        button_text = "Path A"

    adjusted_point_start = (
        int(clicked_points[0][0]) - CENTER_X, int(clicked_points[0][1]) - CENTER_Y)
    adjusted_point_end = (
        int(clicked_points[1][0]) - CENTER_X, int(clicked_points[1][1]) - CENTER_Y)
    path = [adjusted_point_start] + path + [adjusted_point_end]

    for i in range(len(path) - 1):
        adjusted_start = (
            CENTER_X + int(path[i][0]), CENTER_Y + int(path[i][1]))
        adjusted_end = (
            CENTER_X + int(path[i + 1][0]), CENTER_Y + int(path[i + 1][1]))
        new_line = (adjusted_start, adjusted_end)
        lines.append(new_line)  # Append the new line to the lines list
        pygame.draw.line(window, LINE_COLOR, new_line[0], new_line[1], 2)
    return lines, button_text


def handle_draw(lines, points, clicked_images, window, font):
    for line in lines:
        pygame.draw.line(window, LINE_COLOR, line[0], line[1], 2)

    for point in points:
        adjusted_point = (CENTER_X + point[0], CENTER_Y + point[1])
        pygame.draw.circle(window, POINT_COLOR, adjusted_point, POINT_RADIUS)

        label_text = f"({point[0]:.2f}, {point[1]:.2f})"
        label = font.render(label_text, True, LABEL_COLOR)
        window.blit(label, (adjusted_point[0] + 10, adjusted_point[1] - 10))

    for image, x, y in clicked_images:
        window.blit(image, (x - image.get_width() //
                    2, y - image.get_height() // 2))


def draw_button(window, font, text, x, y, width, height, button_color, text_color):
    pygame.draw.rect(window, button_color, (x, y, width, height))
    text_surface = font.render(text, True, text_color)
    text_rect = text_surface.get_rect(center=(x + width // 2, y + height // 2))
    window.blit(text_surface, text_rect)

##### 1.2 Checking Line Intersection

In this exercise, you will complete a function called do_lines_intersect to determine whether two line segments intersect. The function should take four points as input, where each point is represented as a tuple of its (x, y) coordinates: p1, q1 for the first line segment and p2, q2 for the second line segment. Your task is to use the orientation method and the on-segment method to check for the intersection.

Within this function you'll need to define two more functions: the orientation function and the on-segment function. The orientation function is used to determine the relative orientations of the points.

It takes three points and returns:
- 0 if the points are collinear,
- 1 if the orientation is clockwise,
- 2 if the orientation is counterclockwise.

The on-segment function checks if a points line on the segment provided.

For the main function, you need to:

Use the orientation function to determine the relative orientations of the points.
Use the on-segment function to check for special cases where the points are collinear.

Your function should:
- Return True if the line segments intersect.
- Return False if the line segments do not intersect.

In [16]:
def do_lines_intersect(p1, q1, p2, q2):
    pass


##### 1.3 Find Nearest Point

In this exercise, you will complete a function called find_nearest_point to find the nearest point to a given target point from a list of points. The function should take three arguments: target_point, a tuple representing the coordinates of the target point; points, a list of tuples representing the coordinates of the other points; and an optional argument center with a default value of 500. Your task is to calculate the distance between the target point and each point in the list, and return the point that is closest to the target point.

In [12]:
def find_nearest_point(target_point, points, center=500):
    pass

##### 1.4 Get Path Points

In this exercise, you will implement a function called get_path_points that retrieves a sequence of points between two given points from a list of points. The function should take three arguments: start, a tuple representing the starting point; end, a tuple representing the ending point; and points, a list of tuples representing various points. Your task is to handle circular indexing in case the sequence wraps around the end of the list.

You'll need to: 
1. Find Indices
2. Handle Invalid Indices
3. Determine the Path: Is the clockwise or counterclockwise path shorter?
4. Return the Path



In [17]:
def get_path_points(start, end, points):
    pass


If you've correctly implemented the previous three functions, when you run the code block below it should open a pop up window where you can see the path planning in action. Make sure to do lots of testing to check that the behavior is what you think it should be.

In [18]:
# Main Function DO NOT CHANGE
# Load assets
background_image = pygame.image.load('Field.png')
background_image = pygame.transform.scale(
    background_image, (WINDOW_WIDTH, WINDOW_HEIGHT))
click_image_1 = pygame.image.load('Robot.png')
click_image_1 = pygame.transform.scale(click_image_1, (100, 100))
click_image_2 = pygame.image.load('Ball.png')
click_image_2 = pygame.transform.scale(click_image_2, (100, 100))

pygame.init()

window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption(
    'Path Planning Demo ')

pygame.font.init()
font = pygame.font.SysFont('Arial', 18)

clicked_points = []
lines = []
clicked_images = []

RESET = False
SHOW_BUTTON = False
BUTTON_TEXT = ""

RUNNING = True
while RUNNING:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            RUNNING = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            x, y = pygame.mouse.get_pos()
            clicked_points.append((x, y))
            print(x, y)
            if len(clicked_points) == 1:
                clicked_images.append((click_image_1, x, y))
            elif len(clicked_points) == 2:
                clicked_images.append((click_image_2, x, y))
            if RESET is True:
                lines = []
                clicked_points = []
                RESET = False
                SHOW_BUTTON = False
                clicked_images = []
    window.blit(background_image, (0, 0))

    handle_draw(lines, points, clicked_images, window, font)

    if len(clicked_points) == 2:
        new_line = (clicked_points[0], clicked_points[1])
        intersect = False
        for line in predefined_lines:
            if do_lines_intersect(line[0], line[1], new_line[0], new_line[1]):
                intersect = True
                break
        if intersect:
            lines, BUTTON_TEXT = handle_intersection(
                clicked_points, points, lines, window, LINE_COLOR)
            SHOW_BUTTON = True
        else:
            pygame.draw.line(window, LINE_COLOR, new_line[0], new_line[1], 2)
            lines.append(new_line)
        RESET = True
        clicked_points = []

    if SHOW_BUTTON:
        pygame.draw.rect(window, BUTTON_COLOR, (BUTTON_X,
                         BUTTON_Y, BUTTON_WIDTH, BUTTON_HEIGHT))
        text = font.render(BUTTON_TEXT, True, TEXT_COLOR)
        text_rect = text.get_rect(
            center=(BUTTON_X + BUTTON_WIDTH // 2, BUTTON_Y + BUTTON_HEIGHT // 2))
        draw_button(window, font, BUTTON_TEXT, BUTTON_X, BUTTON_Y,
                    BUTTON_WIDTH, BUTTON_HEIGHT, BUTTON_COLOR, TEXT_COLOR)

    pygame.display.update()

pygame.quit()
sys.exit()

SystemExit: 